Quick extract keywords from text for the pdf for royal society; should probably do OCR and clean the text first instead to avoid the problems inherent in the pdfs (but will always be rather unclean?)


WORKS but rather unclean without better OCR --> consider seeing if possible to clean up the texts?
ALSO NEED to otherwise clean the output csv outputs either way

In [10]:
import os
import re
import csv
import pymupdf

def extract_text_from_pdf(pdf_path):
    """Extract text from a given PDF file."""
    doc = pymupdf.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    doc.close()
    return text

def find_contexts(text, keywords, context_size=1000):
    """Find and extract the context around each keyword in the text (case insensitive)."""
    contexts = []
    for keyword in keywords:
        keyword_pattern = r'\b' + re.escape(keyword.strip()) + r'\b'  #regex for word boundary
        matches = re.finditer(keyword_pattern, text, flags=re.IGNORECASE)
        for match in matches:
            start = max(match.start() - context_size, 0)
            end = min(match.end() + context_size, len(text))
            context = text[start:end].strip()
            contexts.append((keyword.strip(), context))
    return contexts

def save_contexts_to_csv(contexts, output_path):
    """Save extracted contexts to a CSV file."""
    with open(output_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Filename", "Keyword", "Context", "Author", "Title"])
        writer.writerows(contexts)

def load_metadata_csv(metadata_path):
    """Load metadata from a CSV file into a dictionary."""
    metadata = {}
    with open(metadata_path, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            filename = row['pdf_filename']
            author = row['author']
            title = row['title']
            metadata[filename] = {'author': author, 'title': title}
    return metadata

def main(folder_path, output_csv, keywords, metadata_csv):
    all_contexts = []
    metadata = load_metadata_csv(metadata_csv)

    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            print(f"Processing file: {filename}")
            pdf_path = os.path.join(folder_path, filename)
            text = extract_text_from_pdf(pdf_path)
            contexts = find_contexts(text, keywords)
            for keyword, context in contexts:
                if filename in metadata: # getting author and title from the royalsociety_files.csv for readability
                    author = metadata[filename]['author']
                    title = metadata[filename]['title']
                    all_contexts.append((filename, keyword, context, author, title))

    save_contexts_to_csv(all_contexts, output_csv)

if __name__ == "__main__":
    folder_path = "D:/Fact_fiction_corpus/royal society/pdf"
    output_csv = "extracted_contexts.csv"
    metadata_csv = "D:/Fact_fiction_corpus/royal society/royalsociety_files.csv" #for giving extra metadata for ease of reading
    keywords = ["fact", "fiction"]  #more keyword variations here later
    main(folder_path, output_csv, keywords, metadata_csv)


Processing file: rstl_1665_0051.pdf
Processing file: rstl_1665_0052.pdf
Processing file: rstl_1665_0053.pdf
Processing file: rstl_1665_0054.pdf
Processing file: rstl_1665_0055.pdf
Processing file: rstl_1665_0056.pdf
Processing file: rstl_1665_0057.pdf
Processing file: rstl_1665_0042.pdf
Processing file: rstl_1665_0043.pdf
Processing file: rstl_1665_0044.pdf
Processing file: rstl_1665_0050.pdf
Processing file: rstl_1665_0041.pdf
Processing file: rstl_1665_0034.pdf
Processing file: rstl_1665_0025.pdf
Processing file: rstl_1665_0018.pdf
Processing file: rstl_1665_0012.pdf
Processing file: rstl_1665_0068.pdf
Processing file: rstl_1665_0067.pdf
Processing file: rstl_1665_0066.pdf
Processing file: rstl_1665_0065.pdf
Processing file: rstl_1665_0064.pdf
Processing file: rstl_1665_0155.pdf
Processing file: rstl_1665_0063.pdf
Processing file: rstl_1665_0062.pdf
Processing file: rstl_1665_0061.pdf
Processing file: rstl_1665_0060.pdf
Processing file: rstl_1665_0059.pdf
Processing file: rstl_1665_0

In [12]:
#testing the output
import pandas as pd

df = pd.read_csv('extracted_contexts.csv')

for index, row in df.head(10).iterrows():
    print(f"Filename: {row['Filename']}")
    print(f"Keyword: {row['Keyword']}")
    print(f"Author: {row['Author']}")
    print(f"Title: {row['Title']}")
    print(f"Context: {row['Context']}")

    print()

Filename: rstl_1668_0026.pdf
Keyword: fact
Author: D. B.
Title: An extract of a letter, written by D. B. to the publisher, concerning the present declination of the magnetick needle, and the tydes, May 23. 1668
Context: ay of the next year, 
13.
1667 5
 and then they found the Variation increaf 
d about 6. mi­
nutes Wefierly. 
M
 
r ^ 
;
New he promifethto take the Gbfervations in the feme Corn* 
pany and Place, on the 1
 
p
k
fa
n
e
 next followin
drant of 6 foot 
radius,accurately divided into minutes, an
a very good Needle about 8. inches long 5
, and he doubts not, 
but the Obfervations fhall be as exa&ly made, as can be done by 
any Men in any place of the World.
He adds, that he has found in feveral places or Meridians, upon 
his own Obfervatipna, from 2. min. to 7. minuts fome difference. 
In this Table he notes the greateft diftance or difference to be 14* 
minutes >
 and fo taking the 
mean
 f
or the true Variatio
cludes it then
 and there to be juft 1. deg. 27. min, viz, fane